# Data Filtering and Binning

This notebook lets you filter and bin tabular data from ISS and INO images.

For best results, run cells from the top down.

In [ ]:
# Run this cell to import required python packages 
# no additions are needed here. You may skip to the next cell after running this one.
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from ROI_functions_images.fluorescence_functions import data_binning, filter_df
import json

# File Loading

Load and view tabular data from CSV file.

In [ ]:
#Run this cell to load csv file: 
file = r'path\file.csv'     #replace path\file.csv with full path to csv file , keeping the quotation marks.
df = pd.read_csv(file)

In [ ]:
# Run this cell to get a description of the csv file's data
df.describe()

In [ ]:
# Run this cell to plot raw data

# change the entries below to df["title of desired column"] to change plot values
x_data = df["Acceptor_intensity"] / df["Donor_intensity"]
y_data = df["FRET_efficiency"]

# change the entries below to edit plot labels/title
title = 'Donor Fluorophore lifetime vs. A/D Ratio'
x_label = 'A/D Ratio'
y_label = 'Donor Lifetime (s)'


plt.figure()
plt.plot(x_data, y_data, 'o')
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title(title)
plt.show()

# Data Filtering
Raw data usually needs to be filtered before it can be used productively. 

Filter parameters to consider may include: 
- ROI size
- donor lifetime
- mean donor intensity
- acceptor to donor ratio
- FRET efficiency
- etc

Filter operations exclusively include:
- \> (use "gt")
- \< (use "lt")
- \>= (use "ge")
- \<= (use "le")
- == (use "ee")

N.B. The filtered dataframe is called subdf as opposed to df.

In [ ]:
# Run this cell to save filter parameters 
# replace None below with the desired filter parameter
#   ex: to select only ROIs with more than 10 pixels -> replace None after 'ROI_size' with "gt10" 
#   N.B. filter parameters must be surrounded by either single or double quotation marks and all 
#        operators must be written using provided abbreviations.

ISS_filter_params = {
    'ROI_size':None,
    'Donor_T0_intensity':None,
    'Acceptor_T0_intensity':None,
    'Donor_intensity':None,
    'Acceptor_intensity':None,
    'Acceptor/Donor_ratio':None,
    'Donor_lifetime':None,
    'Phasor_G':None,
    'Phasor_S':None,
    'FRET_efficiency':None
}

INO_filter_params = {
    'Well_ID':None,    #Well_ID only supports == ("ee") operations
    'ROI_size':None,
    'T0_intensity':None,
    'Donor_intensity':None,
    'Acceptor_intensity':None,    
    'Spectral_Range1_intensity':None,
    'Spectral_Range2_intensity':None,
    'Spectral_Range3_intensity':None,
    'Acceptor/Donor_ratio':None,
    'Donor_lifetime':None,
    'Phasor_G':None,
    'Phasor_S':None,
    'FRET_efficiency':None
}

In [ ]:
# Run this cell to apply saved filter parameters above to the dataframe.

#change **ISS_filter_parameters to **INO_filter_parameters if working with data from an INO image.
subdf = filter_df(df, **ISS_filter_params)  

In [ ]:
# Run this cell to plot filtered data

# change the entries below to subdf["title of desired column"] to change plot data
x_data = subdf["Acceptor_intensity"] / subdf["Donor_intensity"]
y_data = subdf["FRET_efficiency"]

# change the entries below to edit plot labels/title
title = 'FRET efficiency vs. A/D Ratio'
xlabel = 'A/D Ratio'
ylabel = 'FRET Efficiency (%)'

plt.figure()
plt.plot(x_data, y_data, 'o')
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.title(title)
plt.show()

# Binning

Bin the filtered data and calculate mean, median, standard deviation, and standard error.

Bins can be automatically or manually generated.

In [ ]:
# Run this cell to save custom bin edges

# Fill the list below with desired bin edges
#    :ex x_axis_bins = [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5]  will produce 5 bins of width 1, centered at 0, 1, 2, 3, 4

x_axis_bins = [ ]

In [ ]:
# Run this cell to bin selected data

# Change the entries below to set what gets binned.
x_data = subdf["Acceptor_intensity"] / subdf["Donor_intensity"]
y_data = subdf["FRET_efficiency"]
auto_bins = True                      # if True: bins will be set automatically.
custom_bins = x_axis_bins             # auto_bins must be set to False to use custom bins

#change the value of csv_name to save binned data to the desired location
csv_name = r'ROI_binned_data.csv'


bin_size, y_means, y_medians, y_std_devs, y_std_errs, x_bin_centers, x_means = data_binning(x_data, y_data, auto_bins=auto_bins, bin_edges=custom_bins)

bindf = pd.DataFrame({
    'bin_size':bin_size,
    'y_means':y_means,
    'y_medians':y_medians,
    'y_std_devs':y_std_devs,
    'y_std_errs':y_std_errs,
    'x_bin_centers':x_bin_centers,
    'x_means':x_means
})
bindf.to_csv(csv_name)
bindf

# Curve Fitting

Curve fitting can be applied to binned data.

The default curve to be fitted is:

$$E\% =\frac{[Emax * (I_{acceptor}/I_{donor})^h}{Kd^h + (I_{acceptor}/I_{donor})^h} $$

since $I_{acceptor}/I_{donor}$ is just the acceptor to donor ratio, the equation is:

$$E\% =\frac{[Emax * (A:D)^h}{Kd^h + (A:D)^h} $$

In [ ]:
# Run this cell to calculate curve parameters

#this line removes bins with only 1 or fewer values
bindf = bindf[bindf['bin_size'] > 1]

# Change the entries below to set data for curve fit
x_vals = bindf['x_means']
y_vals = bindf['y_means']
y_errs = bindf['y_std_devs']


def model_function(ratio, Emax, Kd, h):
    return (Emax * ratio**h)/(Kd**h + ratio**h)

popt, pcov = opt.curve_fit(model_function, x_vals, y_vals, sigma=y_errs, maxfev=200000)

print('Emax:', popt[0])
print('Kd:', popt[1])
print('h:', popt[2])

In [ ]:
# Run this cell to plot bins and curve fit

# Change entries below below to change plot data
x_vals = bindf['x_means']
y_vals = bindf['y_means']
y_errs = bindf['y_std_devs']

# Change the entries below to edit plot labels/titles
x_label = 'A/D ratio'
y_label = 'FRET efficiency'
title = 'FRET efficiency vs A/D ratio'


plt.figure()
plt.errorbar(x_vals, y_vals, yerr=y_errs, fmt='ko', capsize=5, capthick=1)
plt.plot(x_vals, model_function(x_vals, popt[0], popt[1], popt[2]))
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title(title)
plt.show()

# Save Configuration

Save the current filtering and binning parameters into a json file to batch process other csv files.

In [ ]:
# Run this cell to save parameters as .json file

# Change ISS_filter_params for INO_filter_params if working with INO files
filter_bin_params = ISS_filter_params

# The parameters below will use the values saved above for automatic or custom binning.
filter_bin_params['auto_bins'] = auto_bins
filter_bin_params['x_axis_bins'] = x_axis_bins

# Change the text in quotation marks to set a custom name for your json file.
#    N.B. the file name must end with .json
file_name = 'filter_and_bin_params.json'


with open(file_name, 'w') as newfile:
    json.dump(filter_bin_params, newfile, indent=4)